Importando babriogecas

In [ ]:
from branca.element import Template, MacroElement
from sqlalchemy import create_engine
from folium import plugins
import geopandas as gpd
import pandas as pd
import numpy as np
import folium

tiles_list = ['Stamen Terrain', 'Stamen Toner', 'Stamen Water Color', 'cartodbpositron', 'cartodbdark_matter']

Configuração do banco

In [ ]:
host = "localhost:5432"
database = "db_sig"
user = 'postgres'
password = 'pedro123'

connection_string = f"postgresql+psycopg2://{user}:{password}@{host}/{database}"

engine = create_engine(connection_string)

Questão 1 - Distinção de objetos por meio de seus atributos qualitativos (nominal ou ordinal) por meio de cores

In [ ]:
sql = "select * from macrorregioes_hidrograficas"
df_macro_hidro = gpd.read_postgis(sql, engine, geom_col='area')

In [ ]:
map_macro_hidro = folium.Map([-15.783333, -47.866667],
                 zoom_start=5,
                 )

gjson_macro_hidro = df_macro_hidro.__geo_interface__

cor_macrorregioes = {
    'AMAZÔNICA': 'purple',
    'TOCANTINS-ARAGUAIA': 'blue',
    'ATLÂNTICO NORDESTE OCIDENTAL': 'red',
    'PARNAÍBA': 'yellow',
    'ATLÂNTICO NORDESTE ORIENTAL': 'pink',
    'SÃO FRANCISCO': 'orange',
    'ATLÂNTICO LESTE': 'green',
    'ATLÂNTICO SUDESTE': 'cyan',
    'ATLÂNTICO SUL': 'magenta',
    'URUGUAI': 'brown',
    'PARANÁ': 'teal',
    'PARAGUAI': 'navy',
}

style = lambda feature: {
    'fillColor': cor_macrorregioes[feature['properties']['nm_macrorh']],
    'color': 'black',
    'fillOpacity': 0.5,
    'weight': 1.0,
}

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title> Macrorregiões Hidrográficas </title>
</head>
<body>
  <div class='legenda' 
      style='position: absolute; 
      z-index:2000; 
      background-color: white;
      padding: 2em; 
      right: 3em; 
      bottom: 3em;'>
    <div class='titulo'>Macrorregiões Hidrográficas</div>
    <div class='itens'>
      <ul>
        <li>
          <span style='background:purple;'></span>AMAZÔNICA
        </li>
        <li>
          <span style='background:blue;'></span>TOCANTINS-ARAGUAIA
        </li>
        <li>
          <span style='background:red;'></span>ATLÂNTICO NORDESTE OCIDENTAL
        </li>
        <li>
          <span style='background:yellow;'></span>PARNAÍBA
        </li>
        <li>
          <span style='background:pink;'></span>ATLÂNTICO NORDESTE ORIENTAL
        </li>
        <li>
          <span style='background:orange;'></span>SÃO FRANCISCO
        </li>
        <li>
          <span style='background:green;'></span>ATLÂNTICO LESTE
        </li>
        <li>
          <span style='background:cyan;'></span>ATLÂNTICO SUDESTE
        </li>
        <li>
          <span style='background:magenta;'></span>ATLÂNTICO SUL
        </li>
        <li>
          <span style='background:brown;'></span>URUGUAI
        </li>
        <li>
          <span style='background:teal;'></span>PARANÁ
        </li>
        <li>
          <span style='background:navy;'></span>PARAGUAI
        </li>
      </ul>
    </div>
  </div>
</body>
</html>
<style type='text/css'>
  .titulo {
    text-align: center;
    color: black;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 20px;
  }
    
  .itens ul {
    margin-bottom: 5px;
    float: left;
    list-style: none;
  }
    
  .itens ul li {
    font-size: 14px;
    list-style: none;
    margin-bottom: 2px;
  }
    
  .itens span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 10px;
  }
</style>
{% endmacro %}"""

macro_element = MacroElement()
macro_element._template = Template(template)

map_macro_hidro.get_root().add_child(macro_element)

obj_macro_hidro = folium.GeoJson(gjson_macro_hidro, name='Macrorregiões hidrográficas', style_function=style)
obj_macro_hidro.add_to(map_macro_hidro)

# Adicionando controlador de camadas
for tile in tiles_list:
    folium.TileLayer(tile, overlay = False, control = True).add_to(map_macro_hidro)
folium.LayerControl().add_to(map_macro_hidro)

map_macro_hidro.save('macrorregioes_hidrograficas.html')
map_macro_hidro

Questão 2 - Distinção de objetos por meio de seus atributos quantitativos (numéricos) por meio de escala de cores (pontos, linhas ou áreas) ou raios (pontos)

In [ ]:
sql = "select * from taxa_mortalidade"
df_taxa_mort = gpd.read_postgis(sql, engine)

In [ ]:
map_taxa_mort = folium.Map([-15.783333, -47.866667],
                  zoom_start=5,
                  
                   )

gjson_taxa_mort = df_taxa_mort.__geo_interface__

folium.Choropleth(
    geo_data=gjson_taxa_mort,
    name="choropleth",
    data=df_taxa_mort,
    columns=["nome", "valor"],
    key_on="feature.properties.nome",
    fill_color="BuPu",
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name=" Taxa de mortalidade atribuída a doenças cardiovasculares, câncer, diabetes ou doenças respiratórias crônicas por região geográfica-2014(%)",  
).add_to(map_taxa_mort)

# Adicionando controlador de camadas
for tile in tiles_list:
    folium.TileLayer(tile, overlay = False, control = True).add_to(map_taxa_mort)
folium.LayerControl().add_to(map_taxa_mort)

map_taxa_mort.save('taxa_mortalidade.html')
map_taxa_mort

Questão 3 - Agrupamento de recursos pontuais ou heatmaps;

In [ ]:
sql = "select * from bovinos"
df_bovinos = gpd.read_postgis(sql, engine, geom_col='geom')

In [ ]:
map_bovinos = folium.Map([-15.783333, -47.866667],
                  zoom_start=5
                   )

df_bovinos['latitude'] = df_bovinos['geom'].y.astype(float)
df_bovinos['longitude'] = df_bovinos['geom'].x.astype(float)

# Usa a quantidade de bovinos como peso
heat_data1 = [[row['latitude'], row['longitude'], row['bovino2008']] for index, row in df_bovinos.iterrows()]

plugins.HeatMap(heat_data1,
        name = 'Quantidade de bovinos em 2008',
        blur=5,
        radius=6).add_to(map_bovinos)

# Adicionando controlador de camadas
for tile in tiles_list:
    folium.TileLayer(tile, overlay = False, control = True).add_to(map_bovinos)
folium.LayerControl().add_to(map_bovinos)

map_bovinos.save('bovinos.html')
map_bovinos

Questão 4 - Exibir pelo menos uma camada que seja resultado de junções espaciais.

In [106]:
sql = "select a.geom from agencias_turismo a, taxa_mortalidade t where st_Within(a.geom, t.geom) and t.nome='Nordeste'"

df_ag_turismo = gpd.read_postgis(sql, engine)


map_ag_turismo = folium.Map([-15.783333, -47.866667],
                  zoom_start=5,              
                   )

gjson_ag_turismo = df_ag_turismo.__geo_interface__

points = folium.features.GeoJson(gjson_ag_turismo)

map_ag_turismo.add_child(points)

map_bovinos.save('agencias_turismo_nordeste.html')
map_ag_turismo